In [54]:
import pandas as pd

# 도매 소매처기준별 소매처 명단 만들기

In [ ]:
excel_file = pd.ExcelFile("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx")
print(excel_file.sheet_names)

['伊藤忠食品', 'コゲツ', '加藤産業', '国分', '三井物産', '三菱食品', '藤徳物産', 'カナカン', 'ヤマキ', '旭食品', '湧川']


In [90]:
df_list = []
for i in range(7):
    df = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx",
                       sheet_name=excel_file.sheet_names[i], skiprows=1).iloc[:, 1:]
    df.columns = ["도매소매처코드", "지역", "PY코드", "SP코드", "소매처명", "소매처코드", "담당자ID", "담당자명"]
    df_list.append(df[["도매소매처코드", "소매처코드", "소매처명", "담당자ID", "담당자명"]])

domae_retail_1 = pd.concat(df_list)
domae_retail_1.to_csv("database/csv/domae_retail_1.csv", index=False)

In [91]:
df_list = []
for i in range(7, 11):
    df = pd.read_excel("/Users/nongshim/Downloads/問屋取引先コード・小売店コード26.01.27.xlsx",
                       sheet_name=excel_file.sheet_names[i], skiprows=1).iloc[:, 1:]
    df.columns = ["도매소매처명", "소매처코드", "소매처명", "담당자ID", "담당자명"]
    df.head()
    df_list.append(df)

domae_retail_2 = pd.concat(df_list)
domae_retail_2.to_csv("database/csv/domae_retail_2.csv", index=False)

# 소매처별 판매처 명단 만들기

In [92]:
master_code = pd.read_excel("/Users/nongshim/Downloads/条件マスターコード一覧26.01.29.xlsx").drop_duplicates()
master_code.columns=["판매처코드", "판매처명", "소매처코드", "소매처명", "담당자ID", "담당자명"]
master_code[["소매처코드", "판매처코드", "판매처명"]].to_csv("database/csv/dist_retail.csv", index=False)

# 소매처별 담당자 명단 만들기

In [89]:
# 슈퍼 담당자 매핑
df = pd.read_excel("/Users/nongshim/Downloads/代表スーパーコード一覧.XLSX").drop_duplicates()
df.columns = ['소매처코드', '소매처명', '담당자ID', '담당자명', 'ID']
df[df["담당자ID"].isin(["N3000025", "N3000013"])]
# o30240021

,소매처코드,소매처명,담당자ID,담당자명,ID
453,6020609,■株式会社日立物流,N3000013,営業管理 営業社員,-
577,6033130,■その他,N3000013,営業管理 営業社員,-
768,6056563,■株式会社JR東日本リテールネット,N3000025,◆ＣＶＳ,-
1430,6196420,■辛メーター株式会社,N3000013,営業管理 営業社員,-


In [88]:
# 최근 1년치 판매처-슈퍼 매핑 + 슈퍼 담당자 매핑 병합
# o30240021
col = ['소매처코드', '소매처명', '담당자ID', '담당자명']
super_user_map = pd.concat([df[col].drop_duplicates(), master_code[col].drop_duplicates()]).sort_values(by='담당자ID',ignore_index=True)
super_user_map["소매처명"] = super_user_map["소매처명"].apply(lambda x: x.replace("■", ""))
super_user_map["소매처명"] = super_user_map["소매처명"].apply(lambda x: x.replace("株式会社", ""))
super_user_map = pd.merge(super_user_map, df[["담당자ID","ID"]].drop_duplicates(), on='담당자ID', how='left')
super_user_map.loc[super_user_map.담당자ID.isin(["N3000013", "N3000025"]), "ID"] = "o30230014"
super_user_map.loc[super_user_map.담당자ID.isin(["N3000013", "N3000025"]), "담당자명"] = "高橋果那"
super_user_map.to_csv("database/csv/retail_user.csv", index=False)

# SAP 기준 소매처별 판매처 명단 만들기

In [93]:
sap_retail = pd.read_excel("/Users/nongshim/Downloads/SAP_retail.XLSX")
sap_retail.head()
sap_retail.columns = ["소매처코드", "소매처명","소매처명2", "판매처코드", "판매처명"]
sap_retail.drop(columns=["소매처명2"], inplace=True)
sap_retail.to_csv("database/csv/sap_retail.csv", index=False)

# SAP 기준 제품 명단 만들기

In [117]:
sap_product = pd.read_excel("/Users/nongshim/Downloads/問屋別商品コ_ド一_(도매별_제품코드).xlsx", skiprows=1).iloc[:, 2:]
sap_product = pd.read_excel("/Users/nongshim/Downloads/최근6년간_실적있는_제품코드.xlsx")
# sap_product.columns = ['제품명', 'SAP제품명', '제품코드', 'JAN코드', 'CSJAN코드', 'ITF코드']
# import re
# sap_product["제품명"] = sap_product["제품명"].apply(lambda x: re.sub(r'\s{2,}', ' ', x) if isinstance(x, str) else x)
# sap_product[["제품코드", "제품명"]].drop_duplicates().to_csv("database/csv/sap_product.csv", index=False)

In [105]:
sap_product[sap_product.JAN코드.isna()]

,제품명,SAP제품명,제품코드,JAN코드,CSJAN코드,ITF코드
187,農心韓ラーメンお楽しみ袋ＮＯ１０カフェ１８０４,農心お楽しみ袋No:10K(SS),901001848,NaN,NaN,NaN
263,農心韓ラーメンお楽しみ袋ＮＯ１０カフェ１８０４,農心お楽しみ袋No:10K(SS),901001848,NaN,NaN,NaN
264,農心ラーメンお楽しみ袋 NO:10,農心お楽しみ袋No:10K(SS),901001848,NaN,NaN,NaN
265,農心 韓ラーメンお楽しみ袋 ＮＯ８,農心お楽しみ袋 NO:8K(AW),901001798,NaN,NaN,NaN
266,農心シ゛ャハ゜ン お楽しみ袋 １袋,農心お楽しみ袋 NO:13R(AW),901002180,NaN,NaN,NaN
280,農心 韓ラーメンお楽しみ袋 ＮＯ８,農心お楽しみ袋 NO:8K(AW),901001798,NaN,NaN,NaN
287,ｗａｎｇ タッカルヒ゛ソース ２２０ｇ,WANG タッカルビソース220G,903001916,NaN,NaN,NaN
288,ｗａｎｇ ハニーマスタート゛ソース ２０５ｇ,WANG ハニーマスタードソース205g,903001919,NaN,NaN,NaN
289,農心シ゛ャハ゜ン チャフ゜チェソース ２４０ｇ,WANG チャプチェソース240G,903001373,NaN,NaN,NaN
293,農心 韓ラーメンお楽しみ袋ＮＯ．１４,農心お楽しみ袋 NO：１４R（AW),903002056,NaN,NaN,NaN


# 단가 명단 만들기

In [139]:
unit_price = pd.read_excel("/Users/nongshim/Downloads/★단가리스트★26.02.10.xls", sheet_name="価格表2306", skiprows=2)
unit_price = unit_price[["商品コード", "商品名", "内容量\n換算値", "問屋　　仕切", "本部長　　ｃｔ", "JANコード　      　　　    (ﾊﾗﾙ→UPC)"]]
unit_price.columns = ["제품코드", "제품명", "제품용량", "시키리", "본부장", "JANCD"]
# unit_price = unit_price.dropna(how='all')
unit_price = unit_price.dropna(subset=["제품코드","시키리", "본부장"])
unit_price.to_csv("database/csv/unit_price.csv", index=False)

In [137]:
unit_price.isna().sum()

제품코드     3
제품명      0
제품용량     1
시키리      0
본부장      0
JANCD    0
dtype: int64

In [138]:
unit_price[unit_price.제품코드.isna()]

,제품코드,제품명,제품용량,시키리,본부장,JANCD
41,NaN,辛ラーメン焼きそば炒めキムチ味1Ｐ,137.0,156.0,118.0,8801043046138
77,NaN,ラッポギ旨辛味2人前 ◆ｼｰﾙ対応,414.0,504.0,338.0,8809832940803
78,NaN,ラッポギジャージャー味)人前 ◆ｼｰﾙ対応,414.0,504.0,338.0,8809832940810


In [135]:
unit_price[unit_price.본부장.isna()]

,제품코드,제품명,제품용량,시키리,본부장,JANCD
67,101002524,辛ラーメン20P(コストコ）,NaN,NaN,NaN,NaN
74,901001584,純正韓国ごま油,100.0,300.0,NaN,8801039935859
95,903001423,トッポギ＆ヌードル(ジャジャン味) 混載800,128.0,210.0,NaN,8809061671059
96,903001421,トッポギ＆ヌードル(旨辛味) 混載800,128.0,210.0,NaN,8809061671035
97,903001422,トッポギ＆ヌードル(ハバネロ味) 混載800,132.0,210.0,NaN,8809061671042
100,NaN,コリコレ韓国鍋セット,212.0,300.0,NaN,NaN
106,903001930,コリコレ韓国鍋セットプデチゲ,210.0,300.0,NaN,4582214637280
107,903001931,コリコレ韓国鍋セットキムチチゲ,210.0,300.0,NaN,4582214637303
108,903001932,コリコレ韓国鍋セットコチュジャンチゲ,240.0,300.0,NaN,4582214637327
110,903001991,コリコレポックム 鉄板タッカルビ,205.0,338.0,NaN,4582214637341


In [109]:
print(len(set(sap_product["제품명"])-set(unit_price["제품명"])))
print(len(set(unit_price["제품명"])-set(sap_product["제품명"])))

1496
109


In [110]:
print(len(set(sap_product["제품코드"])-set(unit_price["제품코드"])))
print(len(set(unit_price["제품코드"])-set(sap_product["제품코드"])))

12
28


In [118]:
print(len(set(sap_product["제품코드"])-set(unit_price["제품코드"])))
print(len(set(unit_price["제품코드"])-set(sap_product["제품코드"])))

213
10


In [122]:
unit_price[~unit_price.제품코드.isna()&unit_price.제품코드.isin(set(unit_price["제품코드"])-set(sap_product["제품코드"]))]


,제품코드,제품명,제품용량,시키리,본부장,JANCD
19,101007352,辛ラーメン焼きそばカップ炒めキムチ味,122.0,185.0,145.0,8801043057059
24,101005160(20)/ 101005164,安城湯麺1P 20×2,125.0,126.0,75.0,8801043157681
42,101006322(40)/101006277（20),イワシカルグクス20入(出荷は20×2),98.0,156.0,118.0,8801043013154
52,101006593,辛ラーメンスパイシーチキン袋4P,480.0,532.0,364.0,8801043025058
55,101007382,辛ラーメン焼きそば(炒めキムチ味)4P,548.0,624.0,472.0,8801043046145
59,10100,(仮)アルラーメン1P ★プサン,NaN,72.0,53.0,8801043029605
60,101007101,(仮)アルラーメン5P ★グミ,550.0,360.0,250.0,8801043029841
89,101006601,イワシカルグクス5P,490.0,780.0,590.0,8801043025089
105,101004766/4872,アングリーノグリカップ(縦)※シール貼/ﾛｼﾞﾊﾞﾝﾄﾞ,62.0,132.0,105.0,8801043056427


# 유사도 매핑

In [1]:
# 슈퍼명 유사도 테스트 (CSV 참고용). 실제 앱(내 담당만 등)은 DB pg_trgm 사용 → 결과가 다를 수 있음.
# 여기서는 difflib으로 유사도 0~1 계산 (앱과 동일한 알고리즘 아님)
import pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import re

def split_name_and_capacity(name):
    """
    "제품명" 등에서 용량(예: １２０Ｇ, 120g, 90ML 등)를 분리해서 (이름, 용량) 튜플로 반환.
    용량이 없으면 (원본, None) 반환.
    단, 단위가 그람(g/ｇ/G/Ｇ)일 때는 숫자만 추출해서 반환.
    """
    # 전각/반각/소문자 등 다양한 표기(숫자+단위) (g, G, ｇ, ML 등)
    # 아래 패턴은 마지막에 등장하는 용량(예: １２０Ｇ, 120g, 90ML, 12コ) 등 추출
    # 'Ｇ'도 포함하도록 단위 집합 확장, 전각 및 반각 대소문자 모두 포함!
    pattern = re.compile(
        r"([０-９0-9]+\.?[０-９0-9]*)([gGｇＧmMｍＭlLリットルﾘｯﾄﾙ個コ袋])\s*$"
    )
    # 전각 숫자를 반각으로 바꾸기 위한 테이블
    to_hankaku = str.maketrans("０１２３４５６７８９", "0123456789")
    name = name.strip()
    m = pattern.search(name)
    if m:
        num = m.group(1).translate(to_hankaku)
        unit = m.group(2)
        # 그람 단위(g, G, ｇ, Ｇ)이면 숫자만 남기고, 아니면 단위 붙여 반환
        if unit in ["g", "G", "ｇ", "Ｇ"]:
            cap = num
        else:
            cap = f"{num}{unit}"
        base = name[: m.start()].strip()
        return base, cap
    return name, None


def similarity(a: str, b: str) -> float:
    """두 문자열 유사도 0~1 (pg_trgm과 비슷한 개념)."""
    a, b = (a or "").strip(), (b or "").strip()
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()

def find_similar_supers(
    query: str,
    csv_path: str,
    col: str,
    top_k: int = 10,
    min_similarity: float = 0.0,
    sub_col: str = None,
    sub_query: str = None,
    sub_min_similarity: float = 0.0,
):
    """
    입력한 슈퍼명과 retail_user.csv의 소매처명(또는 지정 컬럼) 중 유사도 높은 순으로 반환.
    2개 이상의 컬럼 유사도 필터 지원: col과 sub_col을 각각 비교.
    """
    path = Path(csv_path)
    if not path.exists():
        raise FileNotFoundError(f"파일 없음: {path}")
    df = pd.read_csv(path, encoding="utf-8")
    if col not in df.columns:
        raise KeyError(f"컬럼 없음: {col}. 사용 가능: {list(df.columns)}")
    if sub_col and sub_col not in df.columns:
        raise KeyError(f"서브 컬럼 없음: {sub_col}. 사용 가능: {list(df.columns)}")

    query = (query or "").strip()
    sub_query = (sub_query or "").strip() if sub_query is not None else None

    rows = []
    for _, row in df.iterrows():
        # 1차 컬럼(주 컬럼) 유사도 계산
        name = (row.get(col) or "")
        if isinstance(name, float) and pd.isna(name):
            name = ""
        name = str(name).strip()
        primary_score = similarity(query, name)
        # 2차 컬럼(옵션) 유사도 계산
        if sub_col and sub_query is not None:
            sub_value = (row.get(sub_col) or "")
            if isinstance(sub_value, float) and pd.isna(sub_value):
                sub_value = ""
            sub_value = str(sub_value).strip()
            sub_score = similarity(sub_query, sub_value)
        else:
            sub_score = None

        # 1차 필터링 (주 컬럼)
        if primary_score >= min_similarity:
            # 2차 필터링 (서브 컬럼, 없으면 패스)
            if (sub_col and sub_query is not None):
                if sub_score >= sub_min_similarity:
                    row_dict = {**row.to_dict(), f"{col}_similarity": round(primary_score, 4), f"{sub_col}_similarity": round(sub_score, 4)}
                    rows.append(row_dict)
            else:
                row_dict = {**row.to_dict(), "similarity": round(primary_score, 4)}
                rows.append(row_dict)
    # 정렬 기준: 1차는 주 컬럼, 둘 다 있으면 (주, 보조 동시 정렬)
    if sub_col and sub_query is not None:
        rows.sort(key=lambda x: (x[f"{col}_similarity"], x[f"{sub_col}_similarity"]), reverse=True)
    else:
        rows.sort(key=lambda x: x.get("similarity", 0), reverse=True)
    return pd.DataFrame(rows).head(top_k)

In [2]:
INPUT_SUPER_NAME = "岩田産業(株)     本社"   
INPUT_SUPER_NAME = "	全国農業（協組連） エーコープ九州"
top = find_similar_supers(INPUT_SUPER_NAME, csv_path="database/csv/retail_user.csv", col="소매처명", top_k=5, min_similarity=0.2)
print(f"입력: '{INPUT_SUPER_NAME}' → 유사도 상위 {len(top)}건")
display(top)

입력: '	全国農業（協組連） エーコープ九州' → 유사도 상위 5건


,소매처코드,소매처명,담당자ID,담당자명,ID,similarity
0,6125210,全国農業協同組合連合会,N3000011,佐々木俊文,o30240005,0.5000
1,6041618,Ａコープ九州,N3000039,八塚琴音,o30240021,0.4348
2,6047988,生活協同組合ユーコープ,N3000034,佐瀬雄一,o30240012,0.4286
3,6001077,全国農業協同組合連合会高知県本部,N3000032,三澤暢太,o30240006,0.4242
4,6033934,エーコープ宮城,N3000016,田和章一,o30240014,0.4167


In [190]:
INPUT_SUPER_NAME = "農心ジャパン 辛カップラーメン ６８Ｇ"
base_name, capacity = split_name_and_capacity(INPUT_SUPER_NAME)
print(f"이름: '{base_name}', 용량: '{capacity}'")

top = find_similar_supers(base_name, csv_path="database/csv/unit_price.csv", col="제품명",top_k=5, min_similarity=0.2, sub_col="제품용량", sub_query=capacity)
print(f"입력: '{base_name}' → 유사도 상위 {len(top)}건")
display(top)

이름: '農心ジャパン 辛カップラーメン', 용량: '68'
입력: '農心ジャパン 辛カップラーメン' → 유사도 상위 5건


,제품코드,제품명,제품용량,시키리,본부장,JANCD,제품명_similarity,제품용량_similarity
0,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694,0.6957,0.6667
1,101003068,★ハラル辛カップラーメン,68.0,148.0,108.0,★031146270606,0.5926,0.6667
2,101002007,ユッケジャンカップ,86.0,91.0,78.0,8801043010139,0.5000,0.3333
3,101001960,旨激辛ラーメン,65.0,91.0,78.0,4582214631165,0.4545,0.3333
4,101000491,辛ラーメン3P,360.0,378.0,225.0,8801043157506,0.4545,0.2857


In [187]:
INPUT_SUPER_NAME = "農心 辛ラーメン 袋 １２０Ｇ"
base_name, capacity = split_name_and_capacity(INPUT_SUPER_NAME)
print(f"이름: '{base_name}', 용량: '{capacity}'")

top = find_similar_supers(base_name, csv_path="database/csv/unit_price.csv", col="제품명",top_k=5, min_similarity=0.2, sub_col="제품용량", sub_query=capacity)
print(f"입력: '{base_name}' → 유사도 상위 {len(top)}건")
display(top)

이름: '農心 辛ラーメン 袋', 용량: '120'
입력: '農心 辛ラーメン 袋' → 유사도 상위 5건


,제품코드,제품명,제품용량,시키리,본부장,JANCD,제품명_similarity,제품용량_similarity
0,101000461,辛ラーメン1P,120.0,126.0,78.0,8801043150606,0.5882,0.7500
1,101001960,旨激辛ラーメン,65.0,91.0,78.0,4582214631165,0.5882,0.2857
2,101000491,辛ラーメン3P,360.0,378.0,225.0,8801043157506,0.5882,0.2500
3,101001977,辛ラーメン5P,600.0,630.0,345.0,8801043009720,0.5882,0.2500
4,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694,0.5556,0.2857
